# Financial Markets Arbitrage Pipeline

This notebook demonstrates the complete Financial Markets (FM) arbitrage strategy:

1. **Connect** to Polymarket and Deribit APIs
2. **Fetch** live spot prices and implied volatility
3. **Parse** price threshold markets ("Will BTC > $X?")
4. **Calculate** fair values using Black-Scholes
5. **Identify** obvious mispricings (15%+ edge)
6. **Execute** paper trades with hedging info

## Background: Options-Implied Arbitrage

Prediction markets like "Will BTC be above $100k?" are essentially **digital options**.
We can calculate their fair value using Black-Scholes and compare to market prices.

This strategy only flags **OBVIOUS** arbitrage - not small statistical edges:
- Spot already crossed the threshold (should be ~100%)
- Fair value is extreme but market diverges significantly
- 15%+ mispricing (model errors don't matter at this scale)

In [ ]:
# Setup: Add project root to path
import sys
sys.path.insert(0, '..')

import asyncio
from datetime import datetime, timezone, timedelta
from IPython.display import display, HTML
import pandas as pd

# Project imports
from src.adapters import PolymarketAdapter, Market
from src.adapters.deribit_options import DeribitOptionsClient
from src.core.config import Credentials
from src.strategies.financial_markets import (
    FinancialMarketsStrategy,
    FinancialOpportunity,
    black_scholes_digital_call,
    calculate_digital_greeks,
)
from src.execution import PaperTrader, CircuitBreakerManager

print("Imports loaded successfully!")

## Step 1: Connect to Deribit for Live Data

In [ ]:
# Fetch live spot prices and volatility from Deribit
async with DeribitOptionsClient() as client:
    btc_spot = await client.get_spot("BTC")
    eth_spot = await client.get_spot("ETH")
    btc_vol = await client.get_historical_vol("BTC")
    eth_vol = await client.get_historical_vol("ETH")
    
    # Try to get ATM implied vol (may not always be available)
    btc_atm_iv = await client.get_atm_iv("BTC", days_to_expiry=30)
    eth_atm_iv = await client.get_atm_iv("ETH", days_to_expiry=30)

print("Live Market Data:")
print(f"  BTC: ${btc_spot:,.0f} | Historical Vol: {btc_vol:.1%}" + 
      (f" | ATM IV: {btc_atm_iv:.1%}" if btc_atm_iv else ""))
print(f"  ETH: ${eth_spot:,.0f} | Historical Vol: {eth_vol:.1%}" +
      (f" | ATM IV: {eth_atm_iv:.1%}" if eth_atm_iv else ""))

## Step 2: Connect to Polymarket

In [ ]:
# Load credentials and connect
credentials = Credentials.from_env()
adapter = PolymarketAdapter(credentials=credentials)

await adapter.connect()
print(f"Connected to Polymarket: {adapter._client is not None}")

In [ ]:
# Fetch markets
markets = await adapter.get_all_markets(active_only=True, limit=500)
print(f"Fetched {len(markets)} active markets")

## Step 3: Configure Strategy

In [ ]:
# Strategy parameters
MIN_EDGE = 0.15  # 15% - only OBVIOUS mispricings

strategy = FinancialMarketsStrategy(min_edge=MIN_EDGE)

# Set live prices
strategy.set_spot_price("BTC", btc_spot)
strategy.set_spot_price("ETH", eth_spot)

# Use ATM IV if available, otherwise historical vol
strategy.set_implied_vol("BTC", btc_atm_iv or btc_vol)
strategy.set_implied_vol("ETH", eth_atm_iv or eth_vol)

print(f"Strategy configured:")
print(f"  Min edge: {MIN_EDGE:.0%} (only obvious arbitrage)")
print(f"  BTC: ${btc_spot:,.0f} @ {(btc_atm_iv or btc_vol):.1%} vol")
print(f"  ETH: ${eth_spot:,.0f} @ {(eth_atm_iv or eth_vol):.1%} vol")

## Step 4: Parse Price Threshold Markets

In [ ]:
# Find all markets that are price threshold markets
parsed_markets = []

for market in markets:
    parsed = strategy.parse_price_threshold_market(market)
    if parsed:
        asset, threshold, expiry = parsed
        parsed_markets.append({
            'market': market,
            'asset': asset,
            'threshold': threshold,
            'expiry': expiry,
        })

print(f"Found {len(parsed_markets)} price threshold markets out of {len(markets)} total")
print(f"Parse rate: {len(parsed_markets)/len(markets):.1%}")

In [ ]:
# Display parsed markets
if parsed_markets:
    parsed_df = pd.DataFrame([
        {
            'Question': p['market'].question[:50] + '...' if len(p['market'].question) > 50 else p['market'].question,
            'Asset': p['asset'],
            'Threshold': f"${p['threshold']:,.0f}",
            'YES Price': f"{p['market'].yes_price:.1%}",
            'Expiry': p['expiry'].strftime('%Y-%m-%d') if p['expiry'] else 'N/A',
        }
        for p in parsed_markets[:15]
    ])
    display(parsed_df)

## Step 5: Black-Scholes Fair Value Calculation

Let's understand how the fair value is calculated for digital options.

In [ ]:
# Example: Calculate fair value for different scenarios
print("Black-Scholes Digital Call Examples:")
print("=" * 60)

scenarios = [
    # (spot, strike, time_years, vol, description)
    (btc_spot, 100000, 0.25, 0.50, "BTC > $100k in 3 months"),
    (btc_spot, btc_spot * 0.9, 0.08, 0.50, "BTC > 10% below current in 1 month"),
    (btc_spot, btc_spot * 1.2, 0.25, 0.50, "BTC > 20% above current in 3 months"),
    (btc_spot, btc_spot * 0.95, 0.01, 0.50, "BTC > 5% below current in 3 days"),
]

for spot, strike, time, vol, desc in scenarios:
    prob = black_scholes_digital_call(spot, strike, time, vol)
    print(f"{desc}:")
    print(f"  Spot: ${spot:,.0f} | Strike: ${strike:,.0f} | Time: {time*365:.0f}d | Vol: {vol:.0%}")
    print(f"  Fair Value: {prob:.1%}")
    print()

In [ ]:
# Calculate Greeks for understanding risk
print("Digital Option Greeks (sensitivity measures):")
print("=" * 60)

greeks = calculate_digital_greeks(
    spot=btc_spot,
    strike=100000,
    time_to_expiry=0.25,  # 3 months
    volatility=0.50,
)

print(f"BTC > $100k in 3 months (at ${btc_spot:,.0f}):")
print(f"  Delta: {greeks.delta:.6f} (price sensitivity per $1 move)")
print(f"  Gamma: {greeks.gamma:.8f} (delta sensitivity)")
print(f"  Theta: {greeks.theta:.4f} (daily time decay)")
print(f"  Vega:  {greeks.vega:.4f} (per 1% vol change)")

## Step 6: Scan for Arbitrage Opportunities

In [ ]:
# Scan all markets for opportunities
opportunities = strategy.scan(markets)

print(f"Found {len(opportunities)} opportunities with >= {MIN_EDGE:.0%} edge")

In [ ]:
# Display opportunities
if opportunities:
    opp_df = pd.DataFrame([
        {
            'Asset': opp.underlying,
            'Threshold': f"${opp.threshold:,.0f}",
            'Spot': f"${opp.spot_price:,.0f}",
            'Market': f"{opp.market_price:.1%}",
            'Fair': f"{opp.fair_value:.1%}",
            'Edge': f"{opp.edge:+.1%}",
            'Direction': opp.direction,
            'Type': opp.arb_type,
            'Obvious': 'Yes' if opp.is_obvious else 'No',
        }
        for opp in opportunities[:15]
    ])
    display(opp_df)
else:
    print("No opportunities found with current parameters.")
    print(f"Try lowering MIN_EDGE (currently {MIN_EDGE:.0%})")

In [ ]:
# Filter for OBVIOUS arbitrage only
obvious_opps = [opp for opp in opportunities if opp.is_obvious]

print(f"OBVIOUS arbitrage opportunities: {len(obvious_opps)}")
print("(These are the safest - spot already crossed threshold or extreme divergence)")

if obvious_opps:
    for opp in obvious_opps[:5]:
        print(f"\n{opp.underlying} > ${opp.threshold:,.0f}:")
        print(f"  Current spot: ${opp.spot_price:,.0f}")
        print(f"  Market says: {opp.market_price:.1%}")
        print(f"  Fair value:  {opp.fair_value:.1%}")
        print(f"  Edge: {opp.edge:+.1%} -> {opp.direction}")
        print(f"  Why obvious: {opp.arb_type}")

## Step 7: Compare Market Prices to Fair Values

In [ ]:
# Detailed comparison for BTC markets
print(f"BTC Price Threshold Markets Comparison")
print(f"Current BTC Spot: ${btc_spot:,.0f}")
print(f"Implied Vol: {(btc_atm_iv or btc_vol):.1%}")
print("=" * 80)

btc_parsed = [p for p in parsed_markets if p['asset'] == 'BTC']

comparison_data = []
for p in btc_parsed[:20]:
    market = p['market']
    threshold = p['threshold']
    expiry = p['expiry']
    
    fair_value = strategy.calculate_fair_value(
        btc_spot, threshold, expiry, btc_atm_iv or btc_vol
    )
    edge = fair_value - market.yes_price
    
    comparison_data.append({
        'Threshold': f"${threshold:,.0f}",
        'Expiry': expiry.strftime('%Y-%m-%d') if expiry else 'N/A',
        'Poly YES': f"{market.yes_price:.1%}",
        'Fair Value': f"{fair_value:.1%}",
        'Edge': f"{edge:+.1%}",
        'Signal': 'BUY' if edge > 0.05 else ('SELL' if edge < -0.05 else '-'),
    })

if comparison_data:
    comp_df = pd.DataFrame(comparison_data)
    display(comp_df)

## Step 8: Paper Trading with Hedging Info

In [ ]:
# Initialize paper trader
BANKROLL = 5000.0
paper_trader = PaperTrader(initial_balance=BANKROLL)

print(f"Paper trader initialized with ${paper_trader.cash_balance:,.2f}")

In [ ]:
# Execute paper trades on obvious opportunities
trades_executed = 0
TRADE_SIZE_USD = 100.0  # $100 per trade

for opp in obvious_opps[:5]:  # Top 5 obvious opportunities
    # Determine trade parameters
    if opp.direction == "BUY":
        side = "YES"
        price = opp.market_price
    else:
        side = "NO"
        price = 1 - opp.market_price
    
    # Skip if price is too extreme (avoid division by zero)
    if price < 0.01 or price > 0.99:
        print(f"Skipping {opp.underlying} ${opp.threshold:,.0f} - price too extreme ({price:.1%})")
        continue
    
    shares = TRADE_SIZE_USD / price
    
    try:
        trade = await paper_trader.execute_paper_trade(
            market_id=opp.market.id,
            side=side,
            size=shares,
            price=price,
            platform=opp.market.platform,
            question=opp.market.question[:50],
        )
        trades_executed += 1
        
        print(f"Traded: {opp.direction} {opp.underlying} ${opp.threshold:,.0f}")
        print(f"  Side: {side} @ {price:.1%}")
        print(f"  Size: {shares:.1f} shares = ${TRADE_SIZE_USD:.0f}")
        print(f"  Edge: {opp.edge:+.1%}")
        if opp.hedge_ratio:
            print(f"  Hedge: {opp.hedge_ratio:+.6f} {opp.underlying} per share")
        print()
        
    except ValueError as e:
        print(f"Trade failed: {e}")

print(f"Executed {trades_executed} paper trades")

In [ ]:
# View portfolio
summary = paper_trader.get_position_summary()

print("Portfolio Summary:")
print(f"  Starting Balance: ${BANKROLL:,.2f}")
print(f"  Cash Balance: ${summary['cash_balance']:,.2f}")
print(f"  Open Positions: {summary['open_positions']}")
print(f"  Total Cost Basis: ${summary['total_cost_basis']:,.2f}")

## Step 9: Greeks and Hedging Analysis

In [ ]:
# Analyze Greeks for traded opportunities
if obvious_opps:
    print("Greeks Analysis for Traded Positions:")
    print("=" * 70)
    
    for opp in obvious_opps[:5]:
        if opp.greeks:
            print(f"\n{opp.underlying} > ${opp.threshold:,.0f}:")
            print(f"  Delta: {opp.greeks.delta:.6f}")
            print(f"  Gamma: {opp.greeks.gamma:.8f}")
            print(f"  Theta: {opp.greeks.theta:.4f} (daily)")
            print(f"  Vega:  {opp.greeks.vega:.4f}")
            
            if opp.hedge_ratio:
                hedge_usd = abs(opp.hedge_ratio) * TRADE_SIZE_USD / opp.market_price * opp.spot_price
                print(f"  Hedge Ratio: {opp.hedge_ratio:+.6f} {opp.underlying}/share")
                print(f"  Hedge Size: ~${hedge_usd:,.0f} of {opp.underlying}")

## Step 10: Cleanup

In [ ]:
# Disconnect from API
await adapter.disconnect()
print("Disconnected from Polymarket API")

## Summary

This notebook demonstrated the complete Financial Markets arbitrage pipeline:

1. **Live Data**: Fetched BTC/ETH spot prices and volatility from Deribit
2. **Market Parsing**: Identified price threshold markets ("Will BTC > $X?")
3. **Fair Value**: Calculated options-implied probabilities using Black-Scholes
4. **Arbitrage Detection**: Found obvious mispricings with 15%+ edge
5. **Paper Trading**: Executed trades with hedging information

### Key Concepts

| Term | Description |
|------|-------------|
| **Digital Option** | Binary payoff ($1 if spot > strike, $0 otherwise) |
| **Fair Value** | Black-Scholes probability that spot > threshold |
| **Edge** | Fair Value - Market Price |
| **Obvious Arb** | Spot already crossed threshold, or extreme divergence |
| **Hedge Ratio** | Delta-based hedge in underlying asset |

### Arbitrage Types

| Type | Description | Safety |
|------|-------------|--------|
| `SPOT_ABOVE_THRESHOLD` | Spot already crossed - should be ~100% | Safest |
| `HIGH_PROBABILITY` | Fair value >90% but market lower | Safe |
| `LOW_PROBABILITY` | Fair value <10% but market higher | Safe |
| `PROBABILITY_EDGE` | Standard Black-Scholes edge | Model-dependent |

### Parameters to Tune

| Parameter | Default | Description |
|-----------|---------|-------------|
| `MIN_EDGE` | 15% | Minimum edge to report (higher = safer) |
| Volatility source | ATM IV | Can use historical or implied vol |

### Risks

1. **Model Risk**: Black-Scholes assumes lognormal prices
2. **Vol Risk**: IV can spike, changing fair values
3. **Execution Risk**: Market may move before trade fills
4. **Liquidity Risk**: May not be able to exit position